In [ ]:
#default_exp layer_features

In [ ]:
#export
from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.callback.all import *
from torchvision.models import vgg16, vgg19
from faststyle import *

In [ ]:
#export
def prepare_feat_model(m):
  m = m.to(default_device()).eval()
  for p in m.parameters(): p.requires_grad=False
  return m

In [ ]:
#export
def get_layers(m, idxs):
  return [m[i] for i in idxs]

In [ ]:
#export
_imagenet_norm = NormalizeAll.from_stats(*imagenet_stats)

In [ ]:
#export
class FeatModels:
  @staticmethod
  def vgg16():
    m = prepare_feat_model(vgg16(True).features)
    stl_ls = get_layers(m, (1, 11, 18, 25))
    cnt_ls = get_layers(m, (20,))
    return dict(m=m, stl_ls=stl_ls, cnt_ls=cnt_ls, tfms=[_imagenet_norm])
  
  @staticmethod
  def vgg19():
    m = prepare_feat_model(vgg19(True).features)
    stl_ls = get_layers(m, (1, 6, 11, 20, 29))
    cnt_ls = get_layers(m, (22,))
    return dict(m=m, stl_ls=stl_ls, cnt_ls=cnt_ls, tfms=[_imagenet_norm])

In [ ]:
#export
class LayerFeats(Module):
  def __init__(self, m, stl_ls, cnt_ls, tfms=None):
    self.m, self.tfms = m, Pipeline(tfms)
    self.stl_hooks = hook_outputs(stl_ls, detach=False)
    self.cnt_hooks = hook_outputs(cnt_ls, detach=False)
  
  def forward(self, x):
    _ = self.m(self.tfms(x))
    return self.stl_hooks.stored, self.cnt_hooks.stored
  
  @classmethod
  def from_feat_m(cls, feat_m): return cls(**feat_m())

In [ ]:
#hide
get_feats = LayerFeats.from_feat_m(FeatModels.vgg19)
tim = TensorImage.create('../examples/styles/abstract.jpg')
stl_fts,cnt_fts = get_feats(tim)

In [ ]:
test_eq(len(stl_fts), 5)
test_eq(len(cnt_fts), 1)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_layer_features.ipynb.
Converted 03_loss.ipynb.
Converted 04_learner.ipynb.
Converted 04_models.ipynb.
Converted 06_callback.ipynb.
